In [1]:
%cd /home/manoelribeiro/PycharmProjects/GeoDiseaseTwitter/
from sklearn.metrics import classification_report
from nnet.models.char_cnn_zhang import CharCNNZhang
from nnet.models.char_cnn_kim import CharCNNKim
from nnet.models.char_tcn import CharTCN
from keras.models import load_model
from nnet.data_utils import Data
import numpy as np
import json

/home/manoelribeiro/PycharmProjects/GeoDiseaseTwitter


Using TensorFlow backend.


In [ ]:
values = ["Campaign", "Personal", "Info.", "Opinion", "Joke"]
nclasses = 5
model_name = "kim"

# Train

In [ ]:
# Load configurations
config = json.load(open("./nnet/config_dengue.json"))

# Load training data
training_data = Data(data_source=config["data"]["training_data_source"], alphabet=config["data"]["alphabet"],
                     input_size=config["data"]["input_size"], num_of_classes=config["data"]["num_of_classes"])
training_data.load_data()
training_inputs, training_labels = training_data.get_all_data()

# Load validation data
validation_data = Data(data_source=config["data"]["validation_data_source"], alphabet=config["data"]["alphabet"],
                       input_size=config["data"]["input_size"], num_of_classes=config["data"]["num_of_classes"])
validation_data.load_data()
validation_inputs, validation_labels = validation_data.get_all_data()

# Load model configurations and build model
if model_name == "kim":
    model = CharCNNKim(input_size=config["data"]["input_size"], alphabet_size=config["data"]["alphabet_size"],
                       num_of_classes=config["data"]["num_of_classes"],
                       **config["char_cnn_kim"])

elif model_name == 'tcn':
    model = CharTCN(input_size=config["data"]["input_size"], alphabet_size=config["data"]["alphabet_size"],
                    num_of_classes=config["data"]["num_of_classes"],
                    **config["char_tcn"])

elif model_name == "zhang":
    model = CharCNNZhang(input_size=config["data"]["input_size"], alphabet_size=config["data"]["alphabet_size"],
                         num_of_classes=config["data"]["num_of_classes"],
                         **config["char_cnn_zhang"])
# Train model
model.train(training_inputs=training_inputs, training_labels=training_labels,
            validation_inputs=validation_inputs, validation_labels=validation_labels,
            epochs=config["training"]["epochs"], batch_size=config["training"]["batch_size"])

# Test

In [ ]:
# Test model
y_score = model.model.predict(validation_inputs)
mask = y_score.max(axis=1,keepdims=1) == y_score
y_pred = np.zeros_like(y_score)
y_pred[mask] = 1

# Print sumary of the imported data
print("Input Shape")
print("-----------")
print("Shape: y_true = " + str(np.shape(validation_labels)))
print("Shape: y_pred = " + str(np.shape(y_score)))
print("--------------------------------------")
print(classification_report(validation_labels, y_pred, target_names=values))

---